In [10]:
import numpy as np
import pandas as pd

In [11]:
pitches = {'Home Run':(1,1),
              'Strike':(2,1),
              'Ball':(3,1),
              'Strike':(4,1),
              'Foul Ball':(5,1),
              'Strike':(6,1),
              'Double':(2,2),
              'Ball':(3,2),
              'Foul Ball':(4,2),
              'Strike':(5,2),
              'Strike':(6,2),
              'Single':(3,3),
              'Fielding Error':(4,3),
              'Strike':(5,3),
              'Ball':(6,3),
              'Hit by Pitch':(4,4),
              'Foul Ball':(5,4),
              'Single':(5,5),
              'Strike':(6,4),
              'Ball':(6,5),
              'Triple':(6,6)
             }

In [60]:
# Build pitch-result lookup dataframe:

r1 = [1,2,3,4,5,6,7,2,3,4,5,6,7,3,4,5,6,7,4,5,6,7,5,6,7,6,7,7]
r2 = [1,1,1,1,1,1,1,2,2,2,2,2,2,3,3,3,3,3,4,4,4,4,5,5,5,6,6,7]
res = ['Home Run!!!!','Strike!','Ball','Strike!','Foul Ball','Strike!','Strike!','Double!!',
       'Ball','Foul Ball','Strike!','Strike!', 'Strike!','Single!','Fielding Error, runner(s) advance','Strike!',
       'Ball','Ball','Hit by Pitch','Foul Ball','Single!','Strike!','Strike!','Ball',
       'Ball','Triple!!!','Ball','Strike!']
hdr = ['Roll_1','Roll_2','Result']

df = pd.DataFrame(np.column_stack([r1,r2,res]), columns=hdr)
df = df.astype({'Roll_1':'int8','Roll_2':'int8','Result':'str'}).copy()

In [4]:
df

,Roll_1,Roll_2,Result
0,1,1,Home Run!!!!
1,2,1,Strike
2,3,1,Ball
3,4,1,Strike
4,5,1,Foul Ball
5,6,1,Strike!
6,7,1,Strike
7,2,2,Double!!
8,3,2,Ball
9,4,2,Foul Ball


In [516]:
class Game():
    def __init__(self, home, visitors):
        self.home_team = home
        self.visiting_team = visitors
        self.batting_team = visitors
        self.fielding_team = home
        self.inning = 1
        self.inning_half = 'top'
        self.balls = 0
        self.strikes = 0
        self.outs = 0
    
    def SetBattingTeam(self, team):
        self.batting_team = team
        
    def GetBattingTeam(self):
        return(self.batting_team)
    
    def SetFieldingTeam(self, team):
        self.fielding_team = team
        
    def GetFieldingTeam(self):
        return(self.fielding_team)
    
    def SetInning(self, i):
        self.inning = i
        
    def GetInning(self):
        return(self.inning)
    
    def SetInningHalf(self, half):
        self.inning_half = half
            
    def GetInningHalf(self):
        return(self.inning_half)
    
    def SetStrikes(self, s):
        self.strikes = s
        
    def GetStrikes(self):
        return(self.strikes)
    
    def SetBalls(self, b):
        self.balls = b 
        
    def GetBalls(self):
        return(self.balls)
        
    def SetOuts(self, o):
        self.outs = o
        
    def GetOuts(self):
        return(self.outs)
    
    def ResetInningStats(self):
        self.inning = self.balls = self.strikes = self.outs = 0
        
    def GetGameState(self):
        return(self.inning, self.balls, self.strikes, self.outs)
    

    # Pitch Result Handlers:
    def HandlePitch(self, p1, p2):
        if(p1 < p2): p1, p2 = p2, p1  # allows combinations, prevents permutations
         
        res = df.loc[(df['Roll_1'].values == p1) & (df['Roll_2'].values == p2)]

        if(res['Result'].values == 'Strike!'):
            self.HandleStrikes()
            
        if(res['Result'].values == 'Ball'):
            self.HandleBalls()
            print('Ball!!!')
            
        if(res['Result'].values == 'Single!') or (res['Result'].values == 'Double!!') or \
        (res['Result'].values == 'Triple!!!') or (res['Result'].values == 'Home Run!!!!'):
            self.HandleHits(res['Result'].values)
 
        if(res['Result'].values == 'Foul Ball'):
            self.HandleFoulBall()
            
        if(res['Result'].values == 'Hit by Pitch'):
            print('Batter hit by pitched ball!!!')
            self.HandleHitByPitchError()
            
        if(res['Result'].values == 'Fielding Error, runner(s) advance'):
            print('Error!!!')
     
    def HandleStrikes(self):
        s = self.GetStrikes()
        if(s < 2):
            self.SetStrikes(s+1)
            print('Strike')
        if(s == 2):
            self.SetStrikes(0)
            self.SetBalls(0)
            self.HandleOuts()
            print('Struck him out swinging!')                  
                  
    def HandleBalls(self):
            b = self.GetBalls()
            if(b < 3):
                self.SetBalls(b+1)
            if(b == 3):
                # check for base runners
                print('Batter walks')
                self.SetBalls(0)
                self.SetStrikes(0)
                  
    def HandleHits(self, r):
        #CheckBases()
        # reset balls and strikes
        self.SetBalls(0)
        self.SetStrikes(0)
        if(r == 'Home Run!!!!'):
            print('It is high! It is far! It is gone! A \'Judgian Blast!\'')
            
        if(r == 'Double!!'):
            print('Double!!!')
        
        if(r == 'Single!'):
            print('Single!!')
            
        if(r == 'Triple!!!'):
            print('Triple!!!')
            
    def HandleFoulBall(self):
        print('Fouled out of play')
        s = self.GetStrikes()
        if(s < 2):
            self.SetStrikes(s+1)
        # add random option for fielder to catch some 
            
    def HandleOuts(self):
            # check number of outs < 2 increment, 2 end inning half, increment inning, reset outs
            # reset balls,strikes, change batting and fielding teams (check for game end)
            outs = self.GetOuts()
            if(outs+1 == 3):
                print('Side retired')
                # change batting and fielding teams
                self.batting_team, self.fielding_team = self.fielding_team, self.batting_team
                self.SetOuts(0)
                
                if(self.GetInningHalf() == 'top'):
                    self.SetInningHalf('bottom')
                else:
                    self.SetInningHalf('top')
                    self.SetInning(self.GetInning()+1)
            else:
                self.SetOuts(self.GetOuts()+1)

    def HandleFieldingError(self):
        # check to see if base runners, if so advance them, call IncrementError, and report
        # else do nothing
        pass
    
    def HandleHitByPitchError(self):
        # check to see if base runners, if so advance them, call MoveRunners, and report
        # Batter to First Base
        self.SetBalls(0)
        self.SetStrikes(0)
    
    def GetCurrentBatterStats(self):
        return( (self.GetBalls(),self.GetStrikes()) )
        
    def GetInningStats(self):
        return( (self.GetOuts(),self.inning_half,self.GetInning()) )


In [517]:
class Team():
    def __init__(self, name, home_visit):
        self.team_name = name
        self.team_home_visit = home_visit
        self.runs = 0
        self.hits = 0
        self.errors = 0
        self.pitcher_strikeouts = 0
        self.pitcher_walks = 0
        
    def SetRuns(self, runs):
        self.runs =+ runs
        
    def SetHits(self, hits):
        self.hits =+ hits
        
    def SetErrors(self, err):
        self.errors =+ err
    
    def GetRuns(self):
        return(self.runs)
    
    def GetHits(self):
        return(self.hits)
    
    def GetErrors(self):
        return(self.errors)
    
    def GetGameStats(self):
        #print('Outs, Runs, Hits, Errors, Men on Base')
        return( (self.runs, self.hits,self.errors) )

In [518]:
# initialize today's game:
home_team = 'Dimes'
visiting_team = 'Pennies'

todays_game = Game(home_team, visiting_team)
Dimes = Team(home_team, 'home')
Pennies = Team(visiting_team, 'visitors')

In [519]:
Pennies.SetHits(1)
Pennies.GetHits()

1

In [1009]:
# Play today's game...

def Pitch():
    p1, p2 = np.random.randint(low=1, high=7, size=2)
    result = todays_game.HandlePitch(p1, p2)
    return(result)

# Main
print(todays_game.GetBattingTeam(), 'are at bat')
results = Pitch()
b,s = todays_game.GetCurrentBatterStats()
o,h,i = todays_game.GetInningStats()

print('Balls:', str(b), '  Strikes:', str(s), '  Outs:', str(o),'  ', h,'of', 'inning:', str(i) )


Pennies are at bat
Side retired
Struck him out swinging!
Balls: 0   Strikes: 0   Outs: 0    bottom of inning: 9


3

In [1010]:
print(sum([0,7,0,5,1,6,13,2,0]))                 # Pennies_Runs
print(sum([2,0,13,2,1,3,13,1]))                   #. Dimes Runs

34
35


In [ ]:
DIMES WIN!